##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Building a RAG Application with Firebase Genkit, Ollama and Gemma



In this comprehensive tutorial, you will learn how to build a **Retrieval-Augmented Generation (RAG)** application using cutting-edge technologies:

[**Genkit**](https://firebase.google.com/docs/genkit) is a framework designed to help you build AI-powered applications and features. It provides open-source libraries for Node.js and Go, plus developer tools for testing and debugging.

[**Gemma**](https://ai.google.dev/gemma) is a family of lightweight, state-of-the-art open language models from Google. Built from the same research and technology used to create the Gemini models, Gemma models are text-to-text, decoder-only large language models (LLMs) available in English, with open weights, pre-trained variants, and instruction-tuned variants.

[**Ollama**](https://ollama.ai/) is a tool that simplifies running language models locally. It allows you to manage and serve multiple models efficiently, making it easier to deploy and test AI models on your machine. With Ollama, you can switch between different models and versions seamlessly, providing flexibility in development and experimentation.

[**Firebase**](https://firebase.google.com/) is a comprehensive app development platform by Google that provides services like real-time databases, authentication, cloud storage, hosting, and machine learning. In this tutorial, you will utilize the **Cloud Firestore**, a scalable, flexible NoSQL cloud database to store and sync data for client- and server-side development.

[**Gradio**](https://gradio.app/) is an open-source Python library for creating user-friendly web interfaces to interact with machine learning models. It allows you to quickly create customizable UI components to interact with your models and also generate shareable web apps that anyone can use.

By integrating these technologies, you will build a powerful RAG application capable of providing accurate and contextually relevant responses based on your custom data.

## What you'll learn

- **Setting Up the Development Environment**: Install and configure Node.js, Genkit, Firebase, Ollama, and Gradio within a Colab notebook.
- **Managing Prompts with Dotprompt**: Modularize your prompts into separate `.prompt` files using **Dotprompt** for better organization and maintainability.
- **Indexing Documents with Genkit Flows**: Use Genkit's flows to embed and index your data, making it retrievable for your RAG application.
- **Building a Chatbot Interface**: Create a user-friendly chatbot interface with Gradio to interact with your app.


Let's get started on building your RAG application!

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/gemma-cookbook/blob/main/Gemma/Gemma_with_Firebase_Genkit_and_Ollama.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>


## **Setup**

Before you begin, make sure you have:

- A basic Google Cloud account.
- Basic knowledge of Node.js and TypeScript.
- Familiarity with Colab notebooks.
- The latest version of **Google Cloud SDK** installed.

## Select the Colab Runtime

In this section, you'll configure Google Colab and set up the tools needed for this project. You'll be using Google Colab as the environment to run your code, so make sure to follow these steps carefully.

1. **Open Google Colab** and create a new notebook.
2. In the upper-right corner of the Colab window, click on the **▾ (Additional connection options)** button.
3. Select **Change runtime type**.
4. Under **Hardware accelerator**, choose **GPU**.
5. Ensure that the **GPU type** is set to **T4**.

This setup will give you enough computing power to run the Gemma model smoothly.

**Once you've completed these steps, you're ready to move on to the next section where you'll set up environment variables in your Colab environment.**

### Configure Your Credentials

First, get your Google API key from: https://aistudio.google.com/app/apikey

You need to set up credentials for Google AI Studio in Google Colab. This allows you to authenticate and securely interact with different services, such as Google AI Studio.


1. Open your Google Colab notebook and click on the 🔑 Secrets tab in the left panel. <img src="https://storage.googleapis.com/generativeai-downloads/images/secrets.jpg" alt="The Secrets tab is found on the left panel." width=50%>
2. **Add Google API Key**:
   - Create a new secret named `GOOGLE_API_KEY`.
   - Paste your Google API Key into the Value input box.
   - Toggle the button to allow notebook access to the secret.

## **Install dependencies**

To build the RAG application, you need to install various tools and libraries. Let's get started with installing the dependencies.

In [ ]:
# Install Gradio
!pip install -q gradio

# Install Ollama
!curl -fsSL https://ollama.ai/install.sh | sh

# Install Node.js
!curl -fsSL https://deb.nodesource.com/setup_20.x | sudo -E bash -
!sudo apt-get install -y nodejs

# Install Genkit CLI and plugins
!npm i -g genkit
!npm i --save genkitx-ollama
!npm i --save @genkit-ai/firebase
!npm i --save @genkit-ai/googleai
!npm i --save @genkit-ai/dotprompt
!npm i llm-chunk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 533.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.6/436.6 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2

## Run Gemma using Ollama

You will use `Ollama` to run the Gemma language model locally. This tool will allow you to interact with the AI and use it in your RAG chatbot application.

First, start the Ollama server. This will run in the background and allow you to call different AI models.

In [ ]:
import subprocess
import time

ollama_serve_process = subprocess.Popen("OLLAMA_KEEP_ALIVE=-1 ollama serve", shell=True)
time.sleep(5)

Ollama provides a library of pre-configured models, including Gemma 2 models. You can browse the available Gemma 2 models at the [Ollama Gemma 2 Model Catalog](https://ollama.com/library/gemma2). This allows you to switch between different Gemma 2 models easily. In this notebook, you'll use the [gemma2:2b](https://ollama.com/library/gemma2:2b) model.

To test if the Gemma model is running correctly, use the following command to ask the model a simple question:

In [ ]:
ollama_run_process = subprocess.Popen(
  "ollama run gemma2:2b 'What is the capital of China?'",
  shell=True, stdout=subprocess.PIPE, text=True
)

print(ollama_run_process.communicate()[0])

The capital of China is **Beijing**. 





You should see the model's response in the output.

##  Set up the Firebase project

Firebase will be used to store and manage your data. In this case, you'll use Cloud Firestore, a NoSQL database that makes it easy to store and retrieve the information that will be used by the chatbot.

Before you can continue, you need to set up a Firebase project:

1.  If you haven't already, create a Firebase project: In the [Firebase console](https://console.firebase.google.com/), click Add project, then follow the on-screen instructions to create a Firebase project or to add Firebase services to an existing GCP project.
<img src="https://i.imgur.com/B8njkTG.png" alt="Welcome to Firebase" width=50%>

2. Then, open your project and go to the **Project settings** page, create a service account and download the service account key file using **Generate new private key**. Keep this file safe, since it grants administrator access to your project.  
<img src="https://i.imgur.com/J20U7lz.png" alt="Project Overview" width=50%>     
<img src="https://i.imgur.com/46FOyMm.png" alt="Service accounts" width=50%>


3. Upload the JSON service account key file and set its location in `GOOGLE_APPLICATION_CREDENTIALS` environmental variable.


In [ ]:
import os
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

  with open('/content/' + fn, 'wb') as f:
    f.write(uploaded[fn])

  os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '/content/' + fn

This will allow you to authenticate to Firebase and use its services.

In [ ]:
# Get the project ID
import json

with open(os.environ["GOOGLE_APPLICATION_CREDENTIALS"], 'r') as f:
    data = json.load(f)
    PROJECT_ID = data['project_id']
    print(PROJECT_ID)

my-genkit-gemma-firebase-demo


## Create a Cloud Firestore database

Now that you've set up Firebase and your development environment, let's move on to creating the Firestore database.

* Navigate to the **Cloud Firestore** section of the [Firebase console](https://console.firebase.google.com/project/_/firestore). You'll be prompted to select an existing Firebase project. Follow the database creation workflow.  
<img src="https://i.imgur.com/WNmcCXa.png" alt="Service accounts" width=50% height=50%>

* To simplify our demo, let's select a starting mode for your **Cloud Firestore Security Rules**. You'll pick **Test mode** to get quickly started.

* Pick a location and this setting will be your project's default Google Cloud Platform (GCP) resource location.

**Note**: Test mode allows open access to your database, which is insecure for production environments. Remember to update your security rules before deploying your application.

## Create a vector index for Firestore

Before you can perform a nearest neighbor search with your vector embeddings, you must create a corresponding index.

To do this, let's first authenticate the Google Cloud SDK to simplify its creation.

In [ ]:
!gcloud auth login --no-launch-browser

Follow the authentication flow in your browser, and copy the authentication code back into the Colab notebook when prompted.

Firestore depends on indexes to provide fast and efficient querying on collections.

**Note**: "index" here refers to database indexes, and not Genkit's indexer and retriever abstractions.

The tutorial requires the embedding field to be indexed to work.

Run the following `gcloud` command as described in the [Firestore docs](https://firebase.google.com/docs/firestore/vector-search?authuser=0#create_and_manage_vector_indexes) to create a single-field vector index.

* `collection-group` is the ID of the collection group.
* `vector-field` is the name of the field that contains the vector embedding.
* `field-config` includes the vector configuration (vector dimension and index type). The dimension is an integer up to 2048. The index type must be flat. You also specify the `field-path` here which is `embedding`.

In [ ]:
%%bash -s "$PROJECT_ID"

# Set the current project ID
gcloud config set project $1

# Create a vector index
gcloud alpha firestore indexes composite create \
  --project=$1 \
  --collection-group=merch \
  --query-scope=COLLECTION \
  --field-config=vector-config='{"dimension":"768","flat": "{}"}',field-path=embedding

## Retrieval-Augmented Generation (RAG)

**Firebase Genkit** provides abstractions that help you build retrieval-augmented generation (RAG) flows, as well as plugins that provide integrations with related tools.

What is RAG?
Retrieval-augmented generation is a technique used to incorporate external sources of information into an LLM’s responses. It's important to be able to do so because, while LLMs are typically trained on a broad body of material, practical use of LLMs often requires specific domain knowledge (for example, you might want to use an LLM to answer customers' questions about your company’s products).

The core Genkit framework offers the abstractions to help you do RAG:

* **Indexers**: add documents to an `"index"`.
* **Embedders**: transforms documents into a vector representation
* **Retrievers**: retrieve documents from an `"index"`, given a query.
These definitions are broad on purpose because Genkit is un-opinionated about what an `"index"` is or how exactly documents are retrieved from it. Genkit only provides a `Document` format and everything else is defined by the retriever or indexer implementation provider.


You'll soon learn how it's possible to ingest a collection of product descriptions into a vector database and retrieve them for use in a flow that determines what items are available. You can even ask general questions about your custom data and Gemma should be able to make sense out of the relevant context that's retrieved from a user query.

###  Genkit Project Setup

Create a new project directory and initialize an NPM project.

In [ ]:
# Create project directory
!mkdir genkit-gemma-sample
%cd genkit-gemma-sample

# Initialize NPM project
!npm init -y

/content/genkit-gemma-sample
Wrote to /content/genkit-gemma-sample/package.json:

{
  "name": "genkit-gemma-sample",
  "version": "1.0.0",
  "main": "index.js",
  "scripts": {
    "test": "echo \"Error: no test specified\" && exit 1"
  },
  "keywords": [],
  "author": "",
  "license": "ISC",
  "description": ""
}



⠙

Initialize a Genkit project and create a sample Ollama project that uses the older Gemma model. You'll update this to use the latest model later.  

In [ ]:
!genkit init --model ollama --non-interactive

### Prepare Data for RAG

Create a file named `products.txt` that contains descriptions of the products you want the chatbot to know about.

In [ ]:
%%writefile products.txt

**Google Pixel 8 Pro - Obsidian - Google Store**

The Google Pixel 8 Pro is Google's latest flagship smartphone, featuring a 6.7-inch LTPO OLED display with a 120Hz refresh rate for smooth and vibrant visuals.
Powered by the Google Tensor G3 chip, it offers exceptional performance, advanced AI capabilities, and enhanced security with the Titan M2 security chip.
The Pixel 8 Pro boasts a versatile triple-camera system, including a 50 MP main sensor, a 48 MP telephoto lens, and a 48 MP ultra-wide lens, enabling you to capture high-quality photos and videos in various lighting conditions.
Innovative camera features like Magic Eraser, Night Sight, and Super Res Zoom enhance your photography experience. The device supports 5G connectivity, has an all-day battery life with fast charging and wireless charging capabilities, and runs on the latest Android OS with guaranteed software updates.

* **Price:** Starting at $999
* **Reviews:** 4.8 out of 5 stars based on customer reviews on the Google Store

---

**Samsung Galaxy Watch5 Pro - 45mm Bluetooth Smartwatch - Black Titanium - Samsung**

The Samsung Galaxy Watch5 Pro is a premium smartwatch designed for outdoor enthusiasts and fitness aficionados.
Featuring a durable Titanium case and Sapphire Crystal Glass, it's built to withstand tough conditions.
The watch includes advanced health monitoring features like ECG, blood pressure measurement, and body composition analysis. It offers GPS route tracking, turn-by-turn navigation, and has a battery life of up to 80 hours.
The Galaxy Watch5 Pro runs on Wear OS Powered by Samsung, providing access to a wide range of apps.

* **Price:** $449.99
* **Reviews:** 4.5 out of 5 stars based on customer reviews on the Samsung website

---

**Dell XPS 13 Laptop - 13.4-inch FHD+ Display - Platinum Silver - Dell**

The Dell XPS 13 is a compact and powerful laptop featuring a 13.4-inch InfinityEdge FHD+ display.
Powered by up to the 11th Gen Intel Core processors, it delivers excellent performance for multitasking and creative work.
The laptop boasts a sleek design with a machined aluminum chassis and a carbon fiber palm rest.
It includes up to 16 GB of RAM and up to 1 TB of SSD storage. With a long battery life and Wi-Fi 6 connectivity, the XPS 13 is ideal for professionals on the go.

* **Price:** Starting at $999.99
* **Reviews:** 4.6 out of 5 stars based on customer reviews on Dell's website

---

**Bose QuietComfort 45 Wireless Noise-Cancelling Headphones - Black - Bose**

The Bose QuietComfort 45 headphones offer world-class noise cancellation with two modes: Quiet and Aware.
They deliver high-fidelity audio with a balanced performance at any volume. The headphones are lightweight and feature synthetic leather ear cushions for all-day comfort.
With up to 24 hours of battery life on a single charge, they are perfect for long flights or extended listening sessions. The headphones support Bluetooth 5.1 for a strong and reliable wireless connection.

* **Price:** $329
* **Reviews:** 4.8 out of 5 stars based on customer reviews on Bose's website

---

**Canon EOS R6 Mirrorless Camera Body - Canon Online Store**

The Canon EOS R6 is a full-frame mirrorless camera designed for both enthusiasts and professionals.
It features a 20.1 MP CMOS sensor and the DIGIC X image processor, providing excellent image quality and high-speed performance.
The camera offers up to 12 fps mechanical shutter and 20 fps electronic (silent) shutter, making it ideal for action photography.
It includes 4K UHD video recording, in-body image stabilization, and Dual Pixel CMOS AF II for fast and accurate autofocus.
The EOS R6 has built-in Wi-Fi and Bluetooth for easy sharing and remote control.

* **Price:** $2,499
* **Reviews:** 4.7 out of 5 stars based on customer reviews on the Canon Online Store

---

**Apple AirPods Pro (2nd Generation) - White - Apple Store**

The Apple AirPods Pro (2nd Generation) offer superior sound quality with Active Noise Cancellation and Adaptive Transparency.
Equipped with the H2 chip, they deliver high-fidelity audio with personalized Spatial Audio features.
The earbuds come with four sizes of silicone ear tips for a customizable fit and include touch controls for media playback and volume adjustment.
With improved battery life, you get up to 6 hours of listening time on a single charge and up to 30 hours with the MagSafe Charging Case.

* **Price:** $249
* **Reviews:** 4.7 out of 5 stars based on customer reviews on the Apple Store

---


Writing products.txt


### Create the Prompt File with **Dotprompt**

Firebase Genkit provides the Dotprompt plugin and text format to help you write and organize your generative AI prompts.

Dotprompt helps you organize and manage the prompts used by the language model. Create a .prompt file to define how the model should interact with the data and users. This makes it easier to maintain and version your prompts, similar to how you manage code.


Create `assistant.prompt` in the `src/prompts` directory.

In [ ]:
# Create a `prompts` directory to store your Dotprompts
!mkdir -p prompts

In [ ]:
%%writefile prompts/assistant.prompt
---
model: ollama/gemma2:2b
config:
  temperature: 0.8
input:
  schema:
    data(array): string
    question: string
output:
  format: text
---
You are acting as a helpful AI assistant that can answer questions using the data that's available.

Use only the context provided to answer the question.
If you don't know, do not make up an answer.

Context:
{{#each data~}}
- {{this}}
{{~/each}}

Question:
{{question}}

Writing prompts/assistant.prompt


### Chunking, Embedding and Indexing

You will then use Genkit to embed and index these product descriptions into Firestore so that the chatbot can retrieve them when answering questions by running the following steps:

* **Chunking**: Next, use `llm-chunk` to break these product descriptions into smaller, manageable chunks. Chunking the data helps ensure that the content is in a suitable size for embedding, making it more effective when working with vector representations. The `llm-chunk` library provides a simple way to split the text into segments that can be vectorized.

* **Embedding**: An embedder is a function that takes content (text, images, audio, etc.) and creates a numeric vector that encodes the semantic meaning of the original content. To populate your Firestore collection, use the `Gecko embeddings` from Google AI along with the `Firebase Admin SDK`.

* **Indexing**: Once the embeddings are created, index them into Firestore so that they can be used later for similarity searches. Store both the text and its embedding in Firestore.

Create `embedFlow.ts` in the `src/flows` directory.

In [ ]:
%%writefile src/embedFlow.ts

import { configureGenkit } from "@genkit-ai/core";
import { embed } from "@genkit-ai/ai/embedder";
import { defineFlow, run } from "@genkit-ai/flow";
import { textEmbeddingGecko001, googleAI } from "@genkit-ai/googleai";
import { FieldValue, getFirestore } from "firebase-admin/firestore";
import { chunk } from "llm-chunk";
import * as z from "zod";
import { readFile } from "fs/promises";
import path from "path";

// Configuration for indexing process
const indexConfig = {
  collection: "merch",  // Firestore collection to store the data
  contentField: "text", // Field name for the text content
  vectorField: "embedding", // Field name for the embedding vector
  embedder: textEmbeddingGecko001, // Embedder model to use
};

// Configure Genkit with Google AI plugin
// Firebase Genkit has a configuration and plugin system.
// Every Genkit app starts with configuration where you specify the plugins
// you want to use and configure various subsystems.
configureGenkit({
  plugins: [googleAI({ apiVersion: ['v1', 'v1beta'] })],
  enableTracingAndMetrics: false,
});

// Initialize Firestore instance
const firestore = getFirestore();

// Create chunking config
// This example uses the llm-chunk library which provides a simple text
// splitter to break up documents into segments that can be vectorized.
const chunkingConfig = {
  minLength: 1000,
  maxLength: 2000,
  splitter: 'sentence',
  overlap: 100,
  //  Split text into chunks using '---' as delimiter
  delimiters: '---',
} as any;

// Define embed flow
export const embedFlow = defineFlow(
  {
    name: "embedFlow", // Name of the flow
    inputSchema: z.void(), // No input is expected
    outputSchema: z.void(), // No output is returned
  },
  async () => {
    // Read text data from file
    const filePath = path.resolve('products.txt');
    const textData = await run("extract-text", () => extractText(filePath));

    // Divide the text into segments.
    const chunks = await run('chunk-it', async () =>
      chunk(textData, chunkingConfig)
    );

    // Index chunks into Firestore.
    await run("index-chunks", async () => indexToFirestore(chunks));
  }
);

// Function to index chunks into Firestore
async function indexToFirestore(data: string[]) {
  for (const text of data) {
    // Generate embedding for the text chunk
    const embedding = await embed({
      embedder: indexConfig.embedder,
      content: text,
    });

    // Add the text and embedding to Firestore
    await firestore.collection(indexConfig.collection).add({
      [indexConfig.vectorField]: FieldValue.vector(embedding),
      [indexConfig.contentField]: text,
    });
  }
}

// Function to read text content from a file
async function extractText(filePath: string) {
  const f = path.resolve(filePath);
  return await readFile(f, 'utf-8');
}

Writing src/embedFlow.ts


By storing both the text and its embedding, you can later perform similarity searches to find relevant product descriptions based on user queries. This makes it possible for the chatbot to retrieve and provide accurate, contextually relevant answers.

### Configuration and plugins

Firebase Genkit has a configuration and plugin system. Every Genkit app starts with configuration where you specify the plugins you want to use and configure various subsystems.

Create `config.ts` in the `src` directory.

In [ ]:
%%writefile src/config.ts

import { configureGenkit } from '@genkit-ai/core';
import { firebase } from '@genkit-ai/firebase';
import { googleAI } from '@genkit-ai/googleai';
import { ollama } from 'genkitx-ollama';
import { dotprompt } from '@genkit-ai/dotprompt';
import { initializeApp, applicationDefault } from 'firebase-admin/app';
import { getFirestore } from 'firebase-admin/firestore';

// Initialize Firebase Admin SDK
const app = initializeApp({
  credential: applicationDefault(),
});

export const firestore = getFirestore(app);

// Configure Genkit
configureGenkit({
  plugins: [
    firebase(),
    googleAI({ apiVersion: ['v1', 'v1beta'] }),
    ollama({
      // Ollama provides an interface to many generative models. Here,
      // you specify Google's Gemma 2 model. The models you specify must already
      // be downloaded and available to the Ollama server.
      models: [{ name: 'gemma2:2b' }],
      // The address of your Ollama API server. This is often a different host
      // from your app backend (which runs Genkit), in order to run Ollama on
      // a GPU-accelerated machine.
      serverAddress: 'http://127.0.0.1:11434',
    }),
    dotprompt(),
  ],
  // Log debug output to tbe console.
  logLevel: 'debug',
  // Perform OpenTelemetry instrumentation and enable trace collection.
  enableTracingAndMetrics: true,
});

Writing src/config.ts


### Defining a RAG Flow

Next, create a flow named `chatbotFlow` that will allow the chatbot to interact with the data you indexed earlier. This flow will combine a retriever (to get relevant information from Firebase) with a prompt that helps format responses. A retriever is a concept that encapsulates logic related to any kind of document retrieval. The most popular retrieval cases typically include retrieval from vector stores; however, in Genkit, a retriever can be any function that returns data. In this case, the retriever is responsible for finding the most relevant product descriptions from Firestore based on the user's question. It uses the **embeddings** and **cosine similarity** to find the closest matches, ensuring that the retrieved information is highly relevant to the query.

Create `memory.ts` and `chatbotFlow.ts` in the `src` directory.

In [ ]:
%%writefile src/memory.ts

import { MessageData } from '@genkit-ai/ai/model';

const chatHistory: Record<string, MessageData[]> = {};

export interface HistoryStore {
  load(id: string): Promise<MessageData[] | undefined>;
  save(id: string, history: MessageData[]): Promise<void>;
}

// You'll also use an in-memory store to store the chat history.
export function inMemoryStore(): HistoryStore {
  return {
    async load(id: string): Promise<MessageData[] | undefined> {
      return chatHistory[id];
    },
    async save(id: string, history: MessageData[]) {
      chatHistory[id] = history;
    },
  };
}

Writing src/memory.ts


In [ ]:
%%writefile src/chatbotFlow.ts

import { defineFlow, run } from '@genkit-ai/flow';
import { defineFirestoreRetriever } from '@genkit-ai/firebase';
import { retrieve } from '@genkit-ai/ai/retriever';
import { textEmbeddingGecko001 } from '@genkit-ai/googleai';
import { z } from 'zod';

import { firestore } from './config';
import { inMemoryStore } from './memory.js';

import { promptRef } from '@genkit-ai/dotprompt';

// Define Firestore retriever
const retrieverRef = defineFirestoreRetriever({
  name: "merchRetriever",
  firestore,
  collection: "merch",  // Collection containing merchandise data
  contentField: "text",  // Field for product descriptions
  vectorField: "embedding", // Field for embeddings
  embedder: textEmbeddingGecko001, // Embedding model
  distanceMeasure: "COSINE", // Similarity metric
});

// Define the prompt reference
const assistantPrompt = promptRef('assistant');

// To store the chat history
const historyStore = inMemoryStore();

// Define chatbot flow
export const chatbotFlow = defineFlow(
  {
    name: "chatbotFlow",
    inputSchema: z.string(),
    outputSchema: z.string(),
  },
  async (question) => {
    const conversationId = '0';

    // Retrieve conversation history.
    const history = await run(
      'retrieve-history',
      conversationId,
      async () => {
        return (await historyStore?.load(conversationId)) || [];
      }
    );

    // Retrieve relevant documents
    const docs = await retrieve({
      retriever: retrieverRef,
      query: question,
      options: { limit: 5 },
    });

    // Run the prompt
    const mainResp = await assistantPrompt.generate({
      history: history,
      input: {
        data: docs.map((doc) => doc.content[0].text || ""),
        question: question,
      },
    });

    // Save history.
    await run(
      'save-history',
      {
        conversationId: conversationId,
        history: mainResp.toHistory(),
      },
      async () => {
        await historyStore?.save(conversationId, mainResp.toHistory());
      }
    );

    // Handle the response from the model API. In this sample, we just convert
    // it to a string, but more complicated flows might coerce the response into
    // structured output or chain the response into another LLM call, etc.
    return mainResp.text();
  }
);

Writing src/chatbotFlow.ts


The retriever works with the LLM (Gemma 2 via Ollama) to create a Retrieval-Augmented Generation (RAG) flow. The retriever fetches relevant documents, which are then used by the language model to generate accurate responses, combining general knowledge with specific, relevant information from your custom data.

The chatbotFlow consists of several key steps:

* **Firestore Retriever**: The `retrieverRef` specifies how to fetch data from Firestore, using fields like `contentField` (product descriptions) and `vectorField` (embeddings) to locate relevant information.

* **Prompt Reference**: The `assistantPrompt` references the prompt you created earlier using Dotprompt, determining how the assistant should format responses.

* **Retrieve and Generate Response**: The chatbot flow retrieves relevant documents and uses them as context to generate a response. It utilizes historical context to provide a coherent and contextually relevant answer.

Finally, wrap up the Genkit app by defining the chatbotFlow and embedFlow in `src/index.ts`. This script starts a flow server that exposes your flows as HTTP endpoints, allowing you to interact with the flows you have defined:

In [ ]:
%%writefile src/index.ts

import { startFlowsServer } from '@genkit-ai/flow';
import { chatbotFlow } from './chatbotFlow';
import { embedFlow } from './embedFlow';

// Start a flow server, which exposes your flows as HTTP endpoints. This call
// must come last, after all of your plug-in configuration and flow definitions.
// You can optionally specify a subset of flows to serve, and configure some
// HTTP server options, but by default, the flow server serves all defined flows.
startFlowsServer({
  flows: [chatbotFlow, embedFlow],
});

Overwriting src/index.ts


### Start the Genkit server

Automatically press `Enter` or `\n` to accept the following terms.


> The Genkit CLI and Developer UI use cookies and similar technologies from Google to deliver and enhance the quality of its services and to analyze usage. Learn more at https://policies.google.com/technologies/cookies

In [ ]:
import os
from google.colab import userdata

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

command = [
    "genkit", "start", "-o", "--port", "8081"
]

# Create a file to write logs
with open("genkit.log", "w") as logfile:
  # Use subprocess.Popen to run the command with nohup-like behavior
  genkit_process = subprocess.Popen(
    command,
    stdout=logfile,
    stderr=subprocess.STDOUT,
    stdin=subprocess.PIPE,
    start_new_session=True  # This is similar to nohup behavior, detaches from terminal
  )
  # Send an Enter key (\n) to the process to accept the terms
  genkit_process.stdin.write(b'\n')
  genkit_process.stdin.flush()

# Sleep for 60 seconds
time.sleep(60)

## Expose the Genkit Tools Web API

Use Colab's proxy to expose the server's Tools API endpoint. You can access the web interface this way in case you need to debug any issues.

In [ ]:
# Uncomment the following code to access the web interface

# from google.colab.output import eval_js
# proxy_url = eval_js("google.colab.kernel.proxyPort(8081)")

# print(f"The Genkit Tools UI is accessible at: {proxy_url}")

## Use `embedFlow` to Index Documents

Now, run `embedFlow` using an HTTP POST curl request to index the documents inside the `.txt` file

In [ ]:
!curl -X POST "http://127.0.0.1:3400/embedFlow" \
  -H "Content-Type: application/json" \
  -d '{}'

You should see a message indicating that the documents have been indexed successfully.

## Use `chatbotFlow` to try the RAG out

Finally, you can query the RAG chatbot to ask some simple questions about your data.

In [ ]:
!curl -X POST "http://127.0.0.1:3400/chatbotFlow" \
  -H "Content-Type: application/json" \
  -d '{"data": "What is the price of the Pixel 8 Pro?"}'

{"result":"The price of the Pixel 8 Pro starts at $999. \n"}

## (Optional) Chat using the Gradio Chatbot Interface

Create a simple web interface using **Gradio**.


In [ ]:
import gradio as gr
import requests


def chat(question, history):
    try:
        response = requests.post(
            "http://127.0.0.1:3400/chatbotFlow",
            headers={"Content-Type": "application/json"},
            json={"data": question}
        )

        # Check for HTTP request errors
        response.raise_for_status()

        json_response = response.json()

        if 'result' in json_response:
            return json_response['result']
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return "Sorry, an unexpected error occurred."


gr.ChatInterface(
  chat,
  chatbot=gr.Chatbot(
    show_copy_button=True,
    elem_id="chatbot",
    render=False,
    render_markdown=True,
    height=300
  ),
  textbox=gr.Textbox(placeholder="Ask me a question", container=False, scale=7),
  title="Firebase Genkit RAG Chatbot",
  description="Ask any question about products.",
  theme="soft",
  examples=[
      {"text": "What is the price of the Pixel 8 Pro?"},
      {"text": "Tell me about the battery life of the Samsung Galaxy Watch5 Pro."},
      {"text": "Does the Google Pixel 8 Pro support 5G connectivity?"}
  ],
  show_progress=True
).launch(debug=True)

This will generate a public URL you can use to access the chatbot interface.

Now that your documents are indexed and the Gradio interface is running, you can start interacting with your RAG application.

Open the Gradio interface using the URL provided and ask questions about the data, such as:

- "What is the price of the Pixel 8 Pro?"
- "Tell me about the battery life of the Samsung Galaxy Watch5 Pro."
- "Does the Google Pixel 8 Pro support 5G connectivity?"

You should receive answers based on the data you provided in the `.txt` file.

## Cleanup

Let's clean up everything as you've approached the end of the tutorial.

In [ ]:
# Terminate all processes
ollama_serve_process.terminate()
ollama_run_process.terminate()
genkit_process.terminate()

# Delete Firebase project (press Y to confirm)
!gcloud projects delete "$PROJECT_ID"

Congratulations! You have successfully built a RAG application using **Genkit**, **Firebase**, **Ollama**, **Gemma**, **Dotprompt**, and **Gradio**, all within a Colab notebook.